In [ ]:
# ===========================================================================
# DIANI Mamoudou Sékou
#
# Avril 2022
#===========================================================================

In [ ]:
# Importation des bibliothèques nécessaires
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.models import load_model
from keras import Model
from sklearn.preprocessing import StandardScaler
from keras import backend as K
from sklearn import svm
from sklearn.model_selection import cross_validate, cross_val_score

# Configuration des GPUs et CPUs
config = tf.compat.v1.ConfigProto(device_count={'GPU': 2, 'CPU': 4})
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess);

In [ ]:
# Chargement du modéle (autoencodeur) sauvegardé dans la section 1 via 1_Modele.py
model_path = "Model.hdf5"
autoencoder = load_model(model_path)

In [ ]:
# Declaration des chemins vers les fichiers cibles
mainDataPath = "vache_elephant/"
datapath = mainDataPath + "test"

# Le nombre des images de test à évaluer
number_images = 400
number_images_class_0 = 200 
number_images_class_1 = 200 

# Les étiquettes des images
labels = np.array([0] * number_images_class_0 +
                  [1] * number_images_class_1)

# La taille des images
image_scale = 96

# La couleur des images
images_color_mode = "rgb"  # grayscale ou rgb

In [ ]:
# Chargement des images test
data_generator = ImageDataGenerator(rescale=1. / 255)

generator = data_generator.flow_from_directory(
    datapath, # Place des images d'entrainement
    color_mode=images_color_mode, # couleur des images
    target_size=(image_scale, image_scale),# taille des images
    batch_size= number_images, # nombre d'images total à charger en mémoire
    class_mode=None,
    shuffle=False) # pas besoin de bouleverser les images

x = generator.next()

In [ ]:
# Reconstruire les images tests en utilisant l'autoencodeur entrainé dans la première étape.
prediction = autoencoder.predict(x)
def show_image(x):
    plt.imshow(np.clip(x + 0.5, 0, 1))
def afficheImg(imgO, imgP, classe):
  plt.subplot(1,3,1)
  plt.title("Original classe "+classe)
  plt.axis('off')
  show_image(imgO)

  plt.subplot(1,3,3)
  plt.title("Reconstructed classe "+classe)
  plt.axis('off')
  show_image(imgP)
  plt.show()

afficheImg(x[0], prediction[0], "0")
afficheImg(x[200], prediction[200], "1")

In [ ]:
# Définition d'un modèle "encoder" qui est formé que de la partie encodeur de l'autoencodeur
input_layer_index = 0 # l'indice de la première couche de l'encodeur
output_layer_index = 9 # l'indice de la dernière couche de l'encodeur 
encoder = Model(autoencoder.layers[0].input, autoencoder.layers[9].output)
embedding = encoder.predict(x)

l=[]
for i in range(len(embedding)):
  a=embedding[i].flatten()
  l.append(a)
embedding=l

# Normalisation de emdedding
scaler = StandardScaler()
embedding = scaler.fit_transform(embedding)

In [ ]:
# Appliquation un SVM Linéaire sur les images originales 
# Entrainement du modèle avec le cross-validation

y=[]
for i in range(number_images_class_0):
  y.append(0)

for i in range(number_images_class_1):
  y.append(1)

nsamples, nx, ny, nz = x.shape
d2_x = x.reshape((nsamples,nx*ny*nz))
lin_svc = svm.LinearSVC() 

cv_results = cross_validate(lin_svc, d2_x, y)
print("l'accuracy: ", cv_results['test_score'])

In [ ]:
# Appliquation un SVC Linéaire sur le flattened embedding normalisé
# Entrainer le modèle avec le cross-validation

nsamples, nx = embedding.shape
d2_x = embedding.reshape((nsamples,nx))
lin_svc = svm.LinearSVC()

cv_results = cross_validate(lin_svc, d2_x, y)
print("l'accuracy: ", cv_results['test_score'])

In [ ]:
# Appliquation d'un TSNE sur le flattened embedding
# affichage des 2D features dans un scatter plot avec des couleurs differentes
# ***********************************************
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2).fit_transform(embedding)

a= X_embedded[0:200]
b= X_embedded[200:400]
x1=[]
y1=[]
x2=[]
y2=[]
for i in range (len(a)):
  x1.append(a[i][0])
  y1.append(a[i][1])

for i in range (len(b)):
  x2.append(b[i][0])
  y2.append(b[i][1])

# Plot
plt.rcParams.update({'figure.figsize':(10,8)})
plt.scatter(x1, y1, label=f'classe 0')
plt.scatter(x2, x2, label=f'classe 1')

# Plot
plt.title('Scatterplot and Correlations')
plt.legend()
plt.show()